# Preparation

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
import os
import sys

In [ ]:
import STAGATE

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "8"

In [ ]:
input_dir = '/home/zbdc/github/Data/Mouse_olfactory_bulb/Slide-seqV2'
counts_file = os.path.join(input_dir, 'Puck_200127_15.digital_expression.txt')
coor_file = os.path.join(input_dir, 'Puck_200127_15_bead_locations.csv')

In [ ]:
counts = pd.read_csv(counts_file, sep='\t', index_col=0)
coor_df = pd.read_csv(coor_file, index_col=0)
print(counts.shape, coor_df.shape)

In [ ]:
adata = sc.AnnData(counts.T)
adata.var_names_make_unique()

In [ ]:
adata.obs_names.to_frame()

In [ ]:
coor_df.index = coor_df['barcode']
coor_df

In [ ]:
merged_df = pd.concat([adata.obs_names.to_frame(), coor_df], axis=1)
merged_df.drop(columns=[0, 'barcode'], inplace=True)
merged_df

In [ ]:
adata.obsm["spatial"] = merged_df.to_numpy()

In [ ]:
sc.pp.calculate_qc_metrics(adata, inplace=True)

In [ ]:
adata

In [ ]:
plt.rcParams["figure.figsize"] = (6,5)
sc.pl.embedding(adata, basis="spatial", color="log1p_total_counts",s=6, show=False)
plt.title('')
plt.axis('off')

In [ ]:
# can be downloaded from https://drive.google.com/drive/folders/10lhz5VY7YfvHrtV40MwaqLmWz56U9eBP?usp=sharing
used_barcode = pd.read_csv(input_dir + 'used_barcodes.txt', sep='\t', header=None)
used_barcode = used_barcode[0]

In [ ]:
adata.obs_names

In [ ]:
adata = adata[used_barcode,]

In [ ]:
plt.rcParams["figure.figsize"] = (5,5)
sc.pl.embedding(adata, basis="spatial", color="log1p_total_counts",s=10, show=False, title='Removing spots outside the main tissue area')

plt.axis('off')

In [ ]:
sc.pp.filter_genes(adata, min_cells=50)
print('After flitering: ', adata.shape)

In [ ]:
#Normalization
sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=3000)
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# Constructing the spatial network

In [ ]:
STAGATE.Cal_Spatial_Net(adata, rad_cutoff=50)
STAGATE.Stats_Spatial_Net(adata)

# Running STAGATE

In [ ]:
adata = STAGATE.train_STAGATE(adata, alpha=0)

In [ ]:
sc.pp.neighbors(adata, use_rep='STAGATE')
sc.tl.umap(adata)

In [ ]:
sc.tl.louvain(adata, resolution=0.5)

In [ ]:
adata.obsm["spatial"] = adata.obsm["spatial"] * (-1)

In [ ]:
plt.rcParams["figure.figsize"] = (3, 3)
sc.pl.embedding(adata, basis="spatial", color="louvain",s=6, show=False, title='STAGATE')
plt.axis('off')

In [ ]:
sc.pl.umap(adata, color='louvain', title='STAGATE')

# SCANPY results (for comparison)

In [ ]:
sc.pp.pca(adata, n_comps=30)

In [ ]:
sc.pp.neighbors(adata, use_rep='X_pca')
sc.tl.louvain(adata, resolution=0.5)
sc.tl.umap(adata)

In [ ]:
plt.rcParams["figure.figsize"] = (3, 3)
sc.pl.embedding(adata, basis="spatial", color="louvain",s=6, show=False, title='SCANPY')
plt.axis('off')

In [ ]:
sc.pl.umap(adata, color='louvain', title='SCANPY')